In [1]:
import pysam
from subprocess import call
import pandas as pd

In [4]:
bam0 = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/bam/m6A/nano_merge_map0.bam"
bam16 = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/bam/m6A/nano_merge_map16.bam"

pre = '/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/tss/nano_merge'
tss_bed = "/mnt/hpc/home/xuxinran/DirectSeq/nanotssqtl/TSSregion.bed"

In [16]:
def get_TSS(bamfile, strand):
    res_dict = set()
    samfile = pysam.AlignmentFile(bamfile, "rb")
    for read in samfile:
        chrom = read.reference_name
        if chrom in [f'chr{i}' for i in range(1, 23)]:
            if strand == '+':
                s = read.reference_start -1
                e = read.reference_start +1
            else:  # strand == '-'
                s = read.reference_end
                e = read.reference_end +2
            res_dict.add((chrom, s, e, read.query_name, 0, strand))
    return res_dict

In [5]:
res_dict_0 = get_TSS(bam0, "+")
res_dict_16 = get_TSS(bam16, "-")

merged_set = res_dict_0.union(res_dict_16)

# 将集合内容写入BED文件
with open(f'{pre}_read5.bed', 'w') as file:
    for item in merged_set:
        file.write('\t'.join(map(str, item)) + '\n')

call(f'bedtools intersect -s -a {pre}_read5.bed -b {tss_bed} -wa -wb> {pre}_overlap.bed',shell=True)
call(f"awk '!seen[$4]++' {pre}_overlap.bed > {pre}_overlap_uniq.bed",shell=True)
call(f'rm {pre}_overlap.bed',shell=True)

0